In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#!unzip "/content/drive/My Drive/img_align_celeba.zip"
!cat '/content/drive/My Drive/utils.py'
!cat '/content/drive/My Drive/ops.py'
!cat '/content/drive/My Drive/models_64x64.py'
import sys
import imageio
sys.path.append('/content/drive/My Drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import re
import scipy
import numpy as np
import tensorflow as tf
import cv2
import imageio

from collections import OrderedDict


def mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        path_dir, _ = os.path.split(path)
        if not os.path.isdir(path_dir):
            os.makedirs(path_dir)


def session(graph=None, allow_soft_placement=True,
            log_device_placement=False, allow_growth=True):
    """ return a Session with simple config """

    config = tf.ConfigProto(allow_soft_placement=allow_soft_placement,
                            log_device_placement=log_device_placement)
    config.gpu_options.allow_growth = allow_growth
    return tf.Session(graph=graph, conf

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import glob
import utils
import traceback
import numpy as np
import tensorflow as tf
import models_64x64 as models

def to_range(images, min_value=0.0, max_value=1.0, dtype=None):
    """
    transform images from [-1.0, 1.0] to [min_value, max_value] of dtype
    """
    assert \
        np.min(images) >= -1.0 - 1e-5 and np.max(images) <= 1.0 + 1e-5 \
        and (images.dtype == np.float32 or images.dtype == np.float64), \
        'The input images should be float64(32) and in the range of [-1.0, 1.0]!'
    if dtype is None:
        dtype = images.dtype
    return ((images + 1.) / 2. * (max_value - min_value) + min_value).astype(dtype)


def batchimwrite(image, path, cnt,prob):
    """ save an [-1.0, 1.0] image """

    for i in range(image.shape[0]):
        image = np.array(image, copy=True)

        if (prob[i] < -0.60 and cnt < 50000) :
          imageio.imwrite("%s_%d.jpg" % (path, cnt) , to_range(image[i, :, :, :], 0, 255, np.uint8))
          cnt += 1
          print(cnt,prob[i])

    return cnt

""" param """
epoch = 50
batch_size = 100
lr = 0.0002
z_dim = 100
gpu_id = 3

''' data '''
# you should prepare your own data in ./data/img_align_celeba
# celeba original size is [218, 178, 3]
tf.reset_default_graph()

""" graphs """
with tf.device('/gpu:%d' % gpu_id):
    ''' models '''
    generator = models.generator
    discriminator = models.discriminator

    ''' graph '''
    # inputs
    z = tf.placeholder(tf.float32, shape=[None, z_dim])

    # generate
    fake = generator(z, training=False ,reuse=False)
    f_logit = discriminator(fake,training=False ,reuse=False)
    


""" train """
''' init '''
# session
sess = utils.session()
# saver
saver = tf.train.Saver()

''' initialization '''
ckpt_dir = '/content/drive/My Drive/GP/checkpoints/celeba_wgan/Epoch_(26)_(542of633).ckpt'
saver.restore(sess, ckpt_dir)
sess.run(tf.global_variables_initializer())
saver.restore(sess, ckpt_dir)

try:
    cnt = 0
    while(True):
        if(cnt >= 50000):
          break
        z_ipt = np.random.normal(size=[batch_size, z_dim])
        img = sess.run(fake, feed_dict={z: z_ipt})
        prob = sess.run(f_logit,{z: z_ipt})
        loss = tf.reduce_mean(prob)
        l = sess.run(loss)
        print(l)
        save_dir = '/content/drive/My Drive/celeba_wgan_results/WGAN50K2'
        utils.mkdir(save_dir + '/')
        cnt = batchimwrite(img, '%s/img' % (save_dir), cnt, prob)
      

except Exception as e:
    traceback.print_exc()
finally:
    print(" [*] Close main session!")
    sess.close()


Streaming output truncated to the last 5000 lines.
45140 [-0.60555446]
45141 [-0.67905885]
45142 [-0.7173649]
45143 [-0.7666696]
45144 [-0.8891041]
45145 [-0.66779524]
45146 [-0.7066226]
45147 [-0.7758456]
45148 [-0.6347861]
45149 [-0.776736]
-0.5445179
45150 [-0.69651794]
45151 [-0.6489409]
45152 [-1.0900753]
45153 [-0.70539606]
45154 [-0.7361433]
45155 [-0.6541312]
45156 [-0.77141654]
45157 [-0.9784248]
45158 [-0.73126286]
45159 [-0.6884028]
45160 [-0.7387675]
45161 [-0.6598575]
45162 [-0.61571765]
45163 [-0.72601354]
45164 [-0.6538005]
45165 [-0.75040203]
45166 [-0.7625632]
45167 [-1.0757064]
45168 [-1.0727937]
45169 [-0.77966356]
45170 [-0.6323875]
45171 [-1.0545504]
45172 [-0.8770686]
45173 [-0.7269465]
45174 [-0.66380477]
45175 [-0.9086656]
45176 [-0.7442814]
45177 [-0.8816537]
45178 [-0.931082]
45179 [-0.8092245]
45180 [-0.743397]
45181 [-0.6931168]
45182 [-0.8816465]
45183 [-0.62417924]
45184 [-0.8712203]
45185 [-0.6271897]
45186 [-0.80435026]
45187 [-0.7358658]
45188 [-0.70460

In [ ]:
!zip -r 'WGAN50K1.zip' '/content/drive/My Drive/celeba_wgan_results/WGAN50K1'

Scanning files .. ................................................
updating: content/drive/My Drive/celeba_wgan_results/WGAN50K1/ (stored 0%)
  adding: content/drive/My Drive/celeba_wgan_results/WGAN50K1/img_49000.jpg (deflated 7%)
  adding: content/drive/My Drive/celeba_wgan_results/WGAN50K1/img_49001.jpg (deflated 8%)
  adding: content/drive/My Drive/celeba_wgan_results/WGAN50K1/img_49002.jpg (deflated 7%)
  adding: content/drive/My Drive/celeba_wgan_results/WGAN50K1/img_49003.jpg (deflated 7%)
  adding: content/drive/My Drive/celeba_wgan_results/WGAN50K1/img_49004.jpg (deflated 7%)
  adding: content/drive/My Drive/celeba_wgan_results/WGAN50K1/img_49005.jpg (deflated 7%)
  adding: content/drive/My Drive/celeba_wgan_results/WGAN50K1/img_49006.jpg (deflated 9%)
  adding: content/drive/My Drive/celeba_wgan_results/WGAN50K1/img_49007.jpg (deflated 7%)
  adding: content/drive/My Drive/celeba_wgan_results/WGAN50K1/img_49008.jpg (deflated 7%)
  adding: content/drive/My Drive/celeba_wgan_res